In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from sklearn.utils import shuffle
from sklearn.impute import KNNImputer
import time
from xgboost import XGBClassifier
import sklearn
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train

In [ ]:
test

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
#shuffle training set
train = shuffle(train)
labels = train['loan_paid']
train

In [ ]:
tmp = (train['annual_income'] - train['annual_income'].mean()) / train['annual_income'].std()
print(tmp.isna().sum())
tmp = np.log10(tmp)
tmp[np.isnan(tmp)] = 0
train['log_norm_annual_income'] = tmp
print(tmp.isna().sum())

tmp = (test['annual_income'] - test['annual_income'].mean()) / test['annual_income'].std()
print(tmp.isna().sum())
tmp = np.log10(tmp)
tmp[np.isnan(tmp)] = 0
test['log_norm_annual_income'] = tmp
print(tmp.isna().sum())

plt.xlabel('Log Norm of Annual Income')
plt.ylabel('Count') 
plt.title('Histogram of Log Norm of Annual Income')
plt.hist(tmp)

In [ ]:
tmp = (train['debt_to_income_ratio'] - train['debt_to_income_ratio'].mean()) / train['debt_to_income_ratio'].std()
print(tmp.isna().sum())
tmp = np.log10(tmp)
tmp[np.isnan(tmp)] = 0
train['log_norm_debt_to_income_ratio'] = tmp
print(tmp.isna().sum())

tmp = (test['debt_to_income_ratio'] - test['debt_to_income_ratio'].mean()) / test['debt_to_income_ratio'].std()
print(tmp.isna().sum())
tmp = np.log10(tmp)
tmp[np.isnan(tmp)] = 0
test['log_norm_debt_to_income_ratio'] = tmp
print(tmp.isna().sum())

plt.xlabel('Log Norm of Debt To Income Ratio')
plt.ylabel('Count') 
plt.title('Histogram of Log Norm of Debt To Income Ratio')
plt.hist(tmp)

In [ ]:
tmp = train['annual_income'] * train['debt_to_income_ratio']
tmp = (tmp - tmp.mean()) / tmp.std()
print(tmp.isna().sum())
tmp = np.log10(tmp)
tmp[np.isnan(tmp)] = 0
train['log_norm_debt'] = tmp
print(tmp.isna().sum())

tmp = test['annual_income'] * test['debt_to_income_ratio']
tmp = (tmp - tmp.mean()) / tmp.std()
print(tmp.isna().sum())
tmp = np.log10(tmp)
tmp[np.isnan(tmp)] = 0
test['log_norm_debt'] = tmp
print(tmp.isna().sum())

plt.xlabel('Log Norm of Debt')
plt.ylabel('Count') 
plt.title('Histogram of Log Norm of Debt')
plt.hist(tmp)

In [ ]:
tmp = (train['total_revolving_limit'] - train['total_revolving_limit'].mean()) / train['total_revolving_limit'].std()
print(tmp.isna().sum())
tmp = np.log10(tmp)
tmp[np.isnan(tmp)] = 0
train['log_norm_total_revolving_limit'] = tmp
print(tmp.isna().sum())

tmp = (test['total_revolving_limit'] - test['total_revolving_limit'].mean()) / test['total_revolving_limit'].std()
print(tmp.isna().sum())
tmp = np.log10(tmp)
tmp[np.isnan(tmp)] = 0
test['log_norm_total_revolving_limit'] = tmp
print(tmp.isna().sum())

plt.xlabel('Log Norm of Total Revolving Limit')
plt.ylabel('Count') 
plt.title('Histogram of Log Norm of Total Revolving Limit')
plt.hist(tmp)

In [ ]:
tmp = (train['requested_amnt'] - train['requested_amnt'].mean()) / train['requested_amnt'].std()
print(tmp.isna().sum())
train['norm_requested_amnt'] = tmp
print(tmp.isna().sum())

tmp = (test['requested_amnt'] - test['requested_amnt'].mean()) / test['requested_amnt'].std()
print(tmp.isna().sum())
test['norm_requested_amnt'] = tmp
print(tmp.isna().sum())

plt.xlabel('Norm of Requested Amnt')
plt.ylabel('Count') 
plt.title('Histogram of Norm of Requested Amnt')
plt.hist(tmp)

In [ ]:
train['fico_score_range'] = ((train['fico_score_range_high'] - train['fico_score_range_low']))
tmp = (train['fico_score_range'] - train['fico_score_range'].mean()) / train['fico_score_range'].std()
print(tmp.isna().sum())
tmp = np.log10(tmp)
tmp[np.isnan(tmp)] = 0
train['log_norm_fico_score_range'] = tmp
print(tmp.isna().sum())

test['fico_score_range'] = ((test['fico_score_range_high'] - test['fico_score_range_low']))
tmp = (test['fico_score_range'] - test['fico_score_range'].mean()) / test['fico_score_range'].std()
print(tmp.isna().sum())
tmp = np.log10(tmp)
tmp[np.isnan(tmp)] = 0
test['log_norm_fico_score_range'] = tmp
print(tmp.isna().sum())

plt.xlabel('Log Norm of fico_score_range')
plt.ylabel('Count') 
plt.title('Histogram of Log Norm of fico_score_range')
plt.xlim([-0.05,0.25])
plt.hist(tmp,bins=10)

In [ ]:
train['fico_score_avg'] = ((train['fico_score_range_high'] + train['fico_score_range_low'])) / 2
lis = []
for i in train['fico_score_avg']:
    if 300 <= i < 580:
        lis.append('Poor')
    elif 580 <= i < 670:
        lis.append('Fair')
    elif 670 <= i < 740:
        lis.append('Good')
    elif 740 <= i < 800:
        lis.append('Very Good')
    elif 800 <= i:
        lis.append('Exceptional')
train['fico_score_avg'] = lis

test['fico_score_avg'] = ((test['fico_score_range_high'] + test['fico_score_range_low'])) / 2
lis = []
for i in test['fico_score_avg']:
    if 300 <= i < 580:
        lis.append('Poor')
    elif 580 <= i < 670:
        lis.append('Fair')
    elif 670 <= i < 740:
        lis.append('Good')
    elif 740 <= i < 800:
        lis.append('Very Good')
    elif 800 <= i:
        lis.append('Exceptional')
test['fico_score_avg'] = lis

plt.xlabel('fico_score_avg labels')
plt.ylabel('Count') 
plt.title('Histogram of fico_score_avg')
plt.hist(train['fico_score_avg'])

In [ ]:
lis = []
for i in train['loan_duration']:
    lis.append(int(i.split()[0]))
train['loan_duration'] = lis

lis = []
for i in test['loan_duration']:
    lis.append(int(i.split()[0]))
test['loan_duration'] = lis

In [ ]:
tmp = ((train['fico_score_range_high'] + train['fico_score_range_low'])) / 2
lis = []
for i in tmp:
    if 300 <= i < 629:
        avg = (28.50/100 + 32.00/100) / 2
        lis.append(avg)
    elif 629 <= i < 689:
        avg = (17.80/100 + 19.90/100) / 2
        lis.append(avg)
    elif 689 <= i < 719:
        avg = (13.50/100 + 15.50/100) / 2
        lis.append(avg)
    elif 719 <= i:
        avg = (10.73/100 + 12.50/100) / 2
        lis.append(avg)
tmp = train['requested_amnt'] * train['loan_duration']/12 * lis
tmp = (tmp - tmp.mean()) / tmp.std()
train['interest'] = tmp
print(tmp.isna().sum())

tmp = ((test['fico_score_range_high'] + test['fico_score_range_low'])) / 2
lis = []
for i in tmp:
    if 300 <= i < 629:
        avg = (28.50/100 + 32.00/100) / 2
        lis.append(avg)
    elif 629 <= i < 689:
        avg = (17.80/100 + 19.90/100) / 2
        lis.append(avg)
    elif 689 <= i < 719:
        avg = (13.50/100 + 15.50/100) / 2
        lis.append(avg)
    elif 719 <= i:
        avg = (10.73/100 + 12.50/100) / 2
        lis.append(avg)
tmp = test['requested_amnt'] * test['loan_duration']/12 * lis
tmp = (tmp - tmp.mean()) / tmp.std()
test['interest'] = tmp
print(tmp.isna().sum())

plt.hist(train['interest'])

In [ ]:
train[train['zipcode'].isnull()]['state']

In [ ]:
train.loc[train['state'] == 'AL']['zipcode']

In [ ]:
tmp = train['zipcode']
lis = []
for st in tmp:
    st = str(st)
    if st[0] == 'n':
        lis.append('3')
        continue
    lis.append(st[0:1])
train['grouped_zipcode'] = lis

tmp = test['zipcode']
lis = []
for st in tmp:
    st = str(st)
    lis.append(st[0:1])
test['grouped_zipcode'] = lis

In [ ]:
#Missing data
print(train['any_tax_liens'].isna().sum())
train['any_tax_liens'] = train['any_tax_liens'].astype(str)
train['any_tax_liens'] = train['any_tax_liens'].replace(['nan'], '0.0')
train['any_tax_liens'] = train['any_tax_liens'].astype(float)
tmp = train.groupby('any_tax_liens',dropna=False).count().reset_index()
tmp['any_tax_liens'] = tmp['any_tax_liens'].fillna('Unknown')

print(test['any_tax_liens'].isna().sum())
test['any_tax_liens'] = test['any_tax_liens'].astype(str)
test['any_tax_liens'] = test['any_tax_liens'].replace(['nan'], '0.0')
test['any_tax_liens'] = test['any_tax_liens'].astype(float)

#style layout 
layout = go.Layout(
    title="any_tax_liens",
    title_x=0.5,
    xaxis=dict(
        title="any_tax_liens"
    ),
    yaxis=dict(
        title="Count"
    ) ) 
fig = go.Figure(go.Bar(x=tmp['any_tax_liens'], y=tmp['loan_paid']),layout=layout)
fig.show()

In [ ]:
#Missing data
print(train['fico_inquired_last_6mths'].isna().sum())
train['fico_inquired_last_6mths'] = train['fico_inquired_last_6mths'].astype(str)
train['fico_inquired_last_6mths'] = train['fico_inquired_last_6mths'].replace(['nan'], '0.0')
tmp = train.groupby('fico_inquired_last_6mths',dropna=False).count().reset_index()
tmp['fico_inquired_last_6mths'] = tmp['fico_inquired_last_6mths'].fillna('Unknown')

print(test['fico_inquired_last_6mths'].isna().sum())
test['fico_inquired_last_6mths'] = test['fico_inquired_last_6mths'].astype(str)
test['fico_inquired_last_6mths'] = test['fico_inquired_last_6mths'].replace(['nan'], '0.0')

#style layout 
layout = go.Layout(
    title="fico_inquired_last_6mths",
    title_x=0.5,
    xaxis=dict(
        title="fico_inquired_last_6mths"
    ),
    yaxis=dict(
        title="Count"
    ) ) 
fig = go.Figure(go.Bar(x=tmp['fico_inquired_last_6mths'], y=tmp['loan_paid']),layout=layout)
fig.show()

In [ ]:
train['delinquency_last_2yrs'] = train['delinquency_last_2yrs'].astype(str)
tmp = train.groupby('delinquency_last_2yrs',dropna=False).count().reset_index()
tmp['delinquency_last_2yrs'] = tmp['delinquency_last_2yrs'].fillna('Unknown')
train['delinquency_last_2yrs'] = train['delinquency_last_2yrs'].astype(float)

test['delinquency_last_2yrs'] = test['delinquency_last_2yrs'].astype(float)

#style layout 
layout = go.Layout(
    title="delinquency_last_2yrs",
    title_x=0.5,
    xaxis=dict(
        title="delinquency_last_2yrs"
    ),
    yaxis=dict(
        title="Count"
    ) ) 
fig = go.Figure(go.Bar(x=tmp['delinquency_last_2yrs'], y=tmp['loan_paid']),layout=layout)
fig.show()

In [ ]:
#Missing data
print(train['public_bankruptcies'].isna().sum())
train['public_bankruptcies'] = train['public_bankruptcies'].astype(str)
train['public_bankruptcies'] = train['public_bankruptcies'].replace(['nan'], 'Unknown')
tmp = train.groupby('public_bankruptcies',dropna=False).count().reset_index()
tmp['public_bankruptcies'] = tmp['public_bankruptcies'].fillna('Unknown')

print(test['public_bankruptcies'].isna().sum())
test['public_bankruptcies'] = test['public_bankruptcies'].astype(str)
test['public_bankruptcies'] = test['public_bankruptcies'].replace(['nan'], 'Unknown')

#style layout 
layout = go.Layout(
    title="Public Bankruptcies",
    title_x=0.5,
    xaxis=dict(
        title="Public Bankruptcies"
    ),
    yaxis=dict(
        title="Count"
    ) ) 
fig = go.Figure(go.Bar(x=tmp['public_bankruptcies'], y=tmp['loan_paid']),layout=layout)
fig.show()

In [ ]:
tmp = train.groupby('grouped_zipcode',dropna=False).count().reset_index()
tmp['grouped_zipcode'] = tmp['grouped_zipcode'].fillna('Unknown')
#style layout 
layout = go.Layout(
    title="Grouped Zipcode",
    title_x=0.5,
    xaxis=dict(
        title="Grouped Zipcode"
    ),
    yaxis=dict(
        title="Count"
    ) ) 
fig = go.Figure(go.Bar(x=tmp['grouped_zipcode'], y=tmp['loan_paid']),layout=layout)
fig.show()

In [ ]:
lis = []
for i in train['employment_length']:
    if i == '10+ years':
        lis.append('10')
    elif i == '2 years':
        lis.append('2')
    elif i == '< 1 year':
        lis.append('0')
    elif i == '3 years':
        lis.append('3')
    elif i == '1 year':
        lis.append('1')
    elif i == '5 years':
        lis.append('5')
    elif i == '4 years':
        lis.append('4')
    elif i == '6 years':
        lis.append('6')
    elif i == '8 years':
        lis.append('8')
    elif i == '7 years':
        lis.append('7')
    elif i == '9 years':
        lis.append('9')
    else:
        lis.append('Unknown')
train['employment_length'] = lis

lis = []
for i in test['employment_length']:
    if i == '10+ years':
        lis.append('10')
    elif i == '2 years':
        lis.append('2')
    elif i == '< 1 year':
        lis.append('0')
    elif i == '3 years':
        lis.append('3')
    elif i == '1 year':
        lis.append('1')
    elif i == '5 years':
        lis.append('5')
    elif i == '4 years':
        lis.append('4')
    elif i == '6 years':
        lis.append('6')
    elif i == '8 years':
        lis.append('8')
    elif i == '7 years':
        lis.append('7')
    elif i == '9 years':
        lis.append('9')
    else:
        lis.append('Unknown')
test['employment_length'] = lis

In [ ]:
#Missing data
print(train['employment_length'].isna().sum())
tmp = train.groupby('employment_length',dropna=False).count().reset_index()
tmp['employment_length'] = tmp['employment_length'].fillna('Unknown')

print(test['employment_length'].isna().sum())

#style layout 
layout = go.Layout(
    title="Employment Length",
    title_x=0.5,
    xaxis=dict(
        title="Employment Length"
    ),
    yaxis=dict(
        title="Count"
    ) ) 
fig = go.Figure(go.Bar(x=tmp['employment_length'], y=tmp['loan_paid']),layout=layout)
fig.show()

In [ ]:
tmp = train.groupby('reason_for_loan',dropna=False).count().reset_index()
tmp['reason_for_loan'] = tmp['reason_for_loan'].fillna('Unknown')
#style layout 
layout = go.Layout(
    title="Reason For Loan",
    title_x=0.5,
    xaxis=dict(
        title="Reason For Loan"
    ),
    yaxis=dict(
        title="Count"
    ) ) 
fig = go.Figure(go.Bar(x=tmp['reason_for_loan'], y=tmp['loan_paid']),layout=layout)
fig.show()

In [ ]:
tmp = train.groupby('employment_verified',dropna=False).count().reset_index()
tmp['employment_verified'] = tmp['employment_verified'].fillna('Unknown')
#style layout 
layout = go.Layout(
    title="Employment Verified",
    title_x=0.5,
    xaxis=dict(
        title="Employment Verified"
    ),
    yaxis=dict(
        title="Count"
    ) ) 
fig = go.Figure(go.Bar(x=tmp['employment_verified'], y=tmp['loan_paid']),layout=layout)
fig.show()

In [ ]:
tmp = train.groupby('state',dropna=False).count().reset_index()
tmp['state'] = tmp['state'].fillna('Unknown')
#style layout 
layout = go.Layout(
    title="State",
    title_x=0.5,
    xaxis=dict(
        title="State"
    ),
    yaxis=dict(
        title="Count"
    ) ) 
fig = go.Figure(go.Bar(x=tmp['state'], y=tmp['loan_paid']),layout=layout)
fig.show()

In [ ]:
tmp = train.groupby('home_ownership_status',dropna=False).count().reset_index()
tmp['home_ownership_status'] = tmp['home_ownership_status'].fillna('Unknown')
#style layout 
layout = go.Layout(
    title="Home Ownership Status",
    title_x=0.5,
    xaxis=dict(
        title="Home Ownership Status"
    ),
    yaxis=dict(
        title="Count"
    ) ) 
fig = go.Figure(go.Bar(x=tmp['home_ownership_status'], y=tmp['loan_paid']),layout=layout)
fig.show()

In [ ]:
tmp = train.groupby('type_of_application',dropna=False).count().reset_index()
tmp['type_of_application'] = tmp['type_of_application'].fillna('Unknown')
#style layout 
layout = go.Layout(
    title="Type Of Application",
    title_x=0.5,
    xaxis=dict(
        title="Type Of Application"
    ),
    yaxis=dict(
        title="Count"
    ) ) 
fig = go.Figure(go.Bar(x=tmp['type_of_application'], y=tmp['loan_paid']),layout=layout)
fig.show()

In [ ]:
cols = ['annual_income', 'debt_to_income_ratio', 'revolving_balance', 'total_revolving_limit', 'requested_amnt', 'fico_score_range_low', 'fico_score_range_high', 'fico_score_range', 'zipcode', 'ID','race','months_since_last_delinq','employment','extended_reason']
train.drop(cols, axis=1, inplace=True)
test.drop(cols, axis=1, inplace=True)

In [ ]:
fcols = ['loan_paid']
train.drop(fcols, axis=1, inplace=True)
test.drop(fcols, axis=1, inplace=True)

In [ ]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]]).rename(columns=lambda x: feature_to_encode + '_' + str(x))
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return res

In [ ]:
features = ['fico_score_avg', 'fico_inquired_last_6mths', 'public_bankruptcies', 'grouped_zipcode', 'loan_duration', 'employment_length', 'reason_for_loan', 'employment_verified', 'state', 'home_ownership_status', 'type_of_application']
for feature in features:
    print(feature)
    train = encode_and_bind(train, feature)
    test = encode_and_bind(test, feature)
    train, test = train.align(test, join='left', axis=1)

In [ ]:
train.shape, test.shape

In [ ]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=test_size, random_state=seed)

In [ ]:
#Initial Cross-Validation
'''
for lr in [0.1,0.3,0.5,0.7,0.9,1]:
    for mcw in [0,0.5,1]:
        for ss in [0.1,0.3,0.5,0.7,0.9]:
            for spw in [0.25,0.30,0.35]:
                model = XGBClassifier(scale_pos_weight=spw,eta=lr,min_child_weight=mcw,subsample=ss)
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                predictions = [round(value) for value in y_pred]
                mcc = matthews_corrcoef(y_test, predictions)
                print(lr,mcw,ss,spw,mcc)
'''

In [ ]:
model = XGBClassifier(scale_pos_weight=0.35)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
mcc = matthews_corrcoef(y_test, predictions)
mcc

In [ ]:
model = XGBClassifier(scale_pos_weight=0.35)
model.fit(train, labels)
y_pred = model.predict(test)
predictions = [round(value) for value in y_pred]

In [ ]:
ids = pd.read_csv('test.csv')
d = {'ID': ids['ID'], 'loan_paid': predictions}
df = pd.DataFrame(data=d)
df.to_csv('out.csv',index=False)